# Day 4 - Lab 2: Generating a CI/CD Pipeline

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Estimated Time:** 75 minutes

**Introduction:**
A robust CI pipeline is the backbone of modern software development. It automatically builds and tests your code every time a change is made, catching bugs early and ensuring quality. In this lab, you will generate all the configuration-as-code artifacts needed to build a professional CI pipeline for our application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load our application code to provide context for the LLM. The AI needs to see our code's imports to generate an accurate `requirements.txt` file.

**Model Selection:**
Models that are good at understanding code and structured data formats like YAML are ideal. `gpt-4.1`, `o3`, or `codex-mini` are strong choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated configuration files.
- `clean_llm_output()`: To clean up the generated text and code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the application code from Day 3 to provide context
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Task:** Before we can build a Docker image, we need a list of our Python dependencies. Prompt the LLM to analyze your application code and generate a `requirements.txt` file.

**Instructions:**
1.  Write a prompt that provides the LLM with the source code of your FastAPI application (`app_code`).
2.  Instruct it to analyze the `import` statements and generate a list of all external dependencies (like `fastapi`, `uvicorn`, `sqlalchemy`). You should also ask it to include `pytest` for testing.
3.  The output should be formatted as a standard `requirements.txt` file.
4.  Save the artifact to the project's root directory.

In [2]:
# TODO: Write a prompt to generate a requirements.txt file.
requirements_prompt = f"""
use app_code as context, analyze the import statements and generate a list of all
external dependencies (like fasapi, uvicorn, sqlalchemy) include pytest for testing,
the output should be formatted as a standard requirements.txt file, save the artifact
to the project's root directory
"""

print("--- Generating requirements.txt ---")
if app_code:
    requirements_content = get_completion(requirements_prompt, client, model_name, api_provider)
    cleaned_reqs = clean_llm_output(requirements_content, language='text')
    print(cleaned_reqs)
    save_artifact(cleaned_reqs, "requirements.txt")
else:
    print("Skipping requirements generation because app code is missing.")

--- Generating requirements.txt ---
To automate this process within your project, follow these steps:

1. **Create the `requirements.txt`**: You can do this manually by creating a new file named `requirements.txt` in the root directory of your project.

2. **Add the Dependencies**: Copy and paste the dependencies list into this file.

3. **Save the File**: Make sure the file is saved in the root directory of your project.

If you want to automate the extraction of dependencies from your code, consider using tools like `pipreqs`, which can generate a `requirements.txt` file based on the imports found in your project. However, be aware that additional manual adjustments might be necessary to refine the list to match your specific needs and versions.

Here is how you could use `pipreqs`:
✅ Successfully saved artifact to: requirements.txt


### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Task:** Generate a multi-stage `Dockerfile` to create an optimized and secure container image for our application.

> **Tip:** Why a multi-stage Dockerfile? The first stage (the 'builder') installs all dependencies, including build-time tools. The final stage copies only the application code and the necessary installed packages. This results in a much smaller, more secure production image because it doesn't contain any unnecessary build tools.

**Instructions:**
1.  Write a prompt asking for a multi-stage `Dockerfile` for a Python FastAPI application.
2.  Specify the following requirements:
    * Use a slim Python base image (e.g., `python:3.11-slim`).
    * The first stage should install dependencies from `requirements.txt`.
    * The final stage should copy the installed dependencies and the application code.
    * The `CMD` should execute the application using `uvicorn`.
3.  Save the generated file as `Dockerfile` in the project's root.

In [3]:
# TODO: Write a prompt to generate a multi-stage Dockerfile.
dockerfile_prompt = """
make a multistage DockerFile for a Python FastAPI application, use a slim Python base image
(e.g, python:3.11-slim), the first stage shuold copy the installed dependencies from
requirements.txt. the final stage should copy the installed dependencies and the application
code, the CMD should execute the application using uvicorn,save the generated file as DockerFile
in the project's root
"""

print("--- Generating Dockerfile ---")
dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
cleaned_dockerfile = clean_llm_output(dockerfile_content, language='dockerfile')
print(cleaned_dockerfile)

if cleaned_dockerfile:
    save_artifact(cleaned_dockerfile, "Dockerfile")

--- Generating Dockerfile ---
# Stage 1: Install dependencies
FROM python:3.11-slim AS builder

# Set environment variables for Python
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Create and set the working directory
WORKDIR /app

# Copy the requirements file
COPY requirements.txt .

# Install the dependencies
RUN python -m pip install --upgrade pip && \
    pip install --no-cache-dir -r requirements.txt

# Stage 2: Build the final image
FROM python:3.11-slim

# Set environment variables for Python
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Create and set the working directory
WORKDIR /app

# Copy the dependencies from the builder stage
COPY --from=builder /usr/local/lib/python3.11/site-packages /usr/local/lib/python3.11/site-packages
COPY --from=builder /usr/local/bin /usr/local/bin

# Copy the application code
COPY . .

# Expose the port the app runs on
EXPOSE 8000

# Command to run the application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80

### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Task:** Generate a complete GitHub Actions workflow file to automate the build and test process.

**Instructions:**
1.  Write a prompt to generate a GitHub Actions workflow file named `ci.yml`.
2.  Specify the following requirements for the workflow:
    * It should trigger on any `push` to the `main` branch.
    * It should define a single job named `build-and-test` that runs on `ubuntu-latest`.
    * The job should have steps to: 1) Check out the code, 2) Set up a Python environment, 3) Install dependencies from `requirements.txt`, and 4) Run the test suite using `pytest`.
3.  Save the generated YAML file to `.github/workflows/ci.yml`.

In [4]:
# TODO: Write a prompt to generate the GitHub Actions ci.yml file.
ci_workflow_prompt = """
generate a Github Actions workflow files named ci.yml, it should trigger on any
push to the main branch, it should define a single job named build-and-test that
runs on ubuntu-latest, the job should have steps to 1) check out the code, 2) set up a Python
environment, 3) install dependencies from the requirements.txt, and 4) run the test suite using
pytest, save the generated YAML file to .github/workflows/ci.yml
"""

print("--- Generating GitHub Actions Workflow ---")
ci_workflow_content = get_completion(ci_workflow_prompt, client, model_name, api_provider)
cleaned_yaml = clean_llm_output(ci_workflow_content, language='yaml')
print(cleaned_yaml)

if cleaned_yaml:
    # Note: The save_artifact helper creates directories as needed.
    save_artifact(cleaned_yaml, ".github/workflows/ci.yml")

--- Generating GitHub Actions Workflow ---
name: CI

on:
  push:
    branches:
      - main

jobs:
  build-and-test:
    runs-on: ubuntu-latest

    steps:
      - name: Check out code
        uses: actions/checkout@v2

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.x'  # Specify the Python version you need

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt

      - name: Run tests
        run: |
          pytest
✅ Successfully saved artifact to: .github/workflows/ci.yml


## Lab Conclusion

Excellent! You have now generated a complete, professional Continuous Integration pipeline using AI. You created the dependency list, the containerization configuration, and the automation workflow, all from simple prompts. This is a powerful demonstration of how AI can automate complex DevOps tasks, allowing teams to build and ship software with greater speed and confidence.

> **Key Takeaway:** AI is a powerful tool for generating 'Configuration as Code' artifacts. By prompting for specific formats like `requirements.txt`, `Dockerfile`, or `ci.yml`, you can automate the creation of the files that define your entire build, test, and deployment processes.